I used this as an opportunity to learn a little bit about object oriented programming. Classes and methods have been moved into support files (scrapers.py and drug_objects.py), but this notebook is where to actually initiate and check scraping.

This version holds sites as major frame work, so a drug object is assumed to be information gleaned only from one site

In [1]:
from bs4 import BeautifulSoup as bf
import requests, re, pickle, time
from collections import defaultdict
import pandas as pd
# from cleaning_refs import med_dur_conversion as med_dur_conversion

In [2]:
from drug_objects import drug, review

In [3]:
from scrapers import WebMD, DrugsDotCom
from scrapers import build_depression_drugs, scrape_parse_reviews, load_soup

#### Drugs.com

In [14]:
filled_drugs_ddc, all_drugs_ddc, generics_list_ddc  = build_depression_drugs('ddc', 'y', 'drug_list_ddc_pg1', 'load', 3)

loaded list
Cymbalta
number of drugs on short list so far: 1
Zoloft
number of drugs on short list so far: 2
Lexapro
number of drugs on short list so far: 3


In [15]:
filled_drugs_ddc[0].attributes

['drugName',
 'site',
 'condition',
 'reviewDate',
 'userName',
 'ageRange_str',
 'gender',
 'role',
 'medDuration_str',
 'effectiveness',
 'ease_of_use',
 'satisfaction',
 'genRating',
 'upVotes']

In [16]:
filled_drugs_ddc[0].build_df()

In [17]:
''' 
Checking data is filling in.  Not all fields will fill becuase of gaps 
in user input or differences between webMD and drugs.com fields
'''
filled_drugs_ddc[0].df

,ageRange,ageRange_str,comment,condition,drugName,ease_of_use,effectiveness,genRating,gender,medDuration,medDuration_str,reviewDate,role,satisfaction,site,upVotes,userName
0,None,None,"""I was taking Cymbalta for 2 1/2 months (along...",depression,Cymbalta,None,None,1.0,None,"[1, 6]","[1, 6]","November 8, 2017",None,None,Drugs_dot_com,2,Anonymous
1,None,None,"""These capsules enhanced made me feel worse. ...",depression,Cymbalta,None,None,1.0,None,"[24, 60]","[24, 60]","November 2, 2017",None,None,Drugs_dot_com,3,Anonymous
2,None,None,"""Took cymbalta for depression for about 6 mont...",depression,Cymbalta,None,None,1.0,None,None,None,"October 13, 2017",None,None,Drugs_dot_com,10,Anonymous
3,None,None,"""I took my first dose of generic Cymbalta last...",depression,Cymbalta,None,None,1.0,None,"[0, 1]","[0, 1]","September 16, 2017",None,None,Drugs_dot_com,11,Anonymous
4,None,None,"""I was put on this for a while. It made me ve...",depression,Cymbalta,None,None,2.0,None,None,None,"August 18, 2017",None,None,Drugs_dot_com,22,Anonymous
5,None,None,"""I've been taking Cymbalta for depression for ...",depression,Cymbalta,None,None,5.0,None,"[24, 60]","[24, 60]","August 10, 2017",None,None,Drugs_dot_com,24,Anonymous
6,None,None,"""Very effective worked immediately""",depression,Cymbalta,None,None,NaN,None,None,None,"July 24, 2017",None,None,Drugs_dot_com,15,Anonymous
7,None,None,"""Ok so I'm a 19y/o female and, After a month a...",depression,Cymbalta,None,None,6.0,None,"[1, 6]","[1, 6]","June 30, 2017",None,None,Drugs_dot_com,14,Anonymous
8,None,None,"""Had panic attacks and major depression. This ...",depression,Cymbalta,None,None,10.0,None,None,None,"June 28, 2017",None,None,Drugs_dot_com,37,Anonymous
9,None,None,"""Just took it at its smallest available dose t...",depression,Cymbalta,None,None,10.0,None,"[0, 1]","[0, 1]","June 23, 2017",None,None,Drugs_dot_com,58,Anonymous


In [18]:
df = filled_drugs_ddc[0].slice_data('genRating', 1.0)

In [22]:
#check 
mini_drug_list = pickle.load(open('drug_list_ddc_pg1.p', 'rb'))
print(mini_drug_list[0].reviews[5].__dict__)

{'drugName': 'Cymbalta', 'site': 'Drugs_dot_com', 'condition': 'depression', 'reviewDate': 'August 10, 2017', 'userName': 'Anonymous', 'ageRange_str': None, 'ageRange': None, 'gender': None, 'role': None, 'medDuration_str': '[24, 60]', 'medDuration': [24, 60], 'effectiveness': None, 'ease_of_use': None, 'satisfaction': None, 'genRating': 5.0, 'comment': '"I\'ve been taking Cymbalta for depression for two and a half years. My doctor titrated me up to 120mg. I was at 120mg for over two and a half years and almost constantly I was severely depressed, severe anxiety, some paranoia, no energy, over reaction, self harm, extreme anger, just to name a few. That\'s not to say I do have some of those problems normally, but not as severe or as constant. So Cymbalta was overlooked as a cause to my worsening issues. Two weeks ago I was admitted into an inpatient hospital to basically save my life. While I was in there I was abruptly taken off, then titrated up to 60mg. I\'m a totally different pers

#### WebMD

In [26]:
# WebMD takes some time to scrape...
filled_list_wmd, all_drugs_wmd, generics_list_md = build_depression_drugs('wmd', 'n', 'drug_list_wmd_mini', num_drugs=3)

loaded list
Cymbalta
number of drugs on short list so far: 1
Lexapro
number of drugs on short list so far: 2
Effexor XR
number of drugs on short list so far: 3


In [27]:
filled_list_wmd[0].attributeDetails

{'ageRange_str': {'[35, 44]',
  '[65, 74]',
  '[55, 64]',
  '[45, 54]',
  '[19, 24]',
  None,
  '[25, 34]',
  '[13, 18]'},
 'condition': {None},
 'drugName': {'Cymbalta'},
 'ease_of_use': {1.0, 2.0, 3.0, 4.0, 5.0},
 'effectiveness': {1.0, 2.0, 3.0, 4.0, 5.0},
 'genRating': {None},
 'gender': {'Femal', 'Female', 'Male', None},
 'medDuration_str': {'[1, 6]',
  '[6, 12]',
  '[0, 1]',
  '[24, 60]',
  '[120, 300]',
  '[60, 120]',
  None,
  '[12, 24]'},
 'reviewDate': {'8/12/2013',
  '11/9/2010',
  '7/28/2010',
  '12/21/2009',
  '11/1/2012',
  '7/15/2010',
  '2/21/2008',
  '6/20/2009',
  '4/6/2012',
  '5/4/2014',
  '1/17/2010',
  '1/26/2011',
  '11/2/2009',
  '11/17/2010',
  '4/24/2008',
  '7/11/2009',
  '7/29/2015',
  '12/9/2012',
  '8/15/2010',
  '6/16/2015',
  '2/18/2010',
  '1/28/2008',
  '7/3/2010',
  '5/22/2008',
  '1/4/2009',
  '7/6/2010',
  '5/12/2009',
  '1/10/2013',
  '7/4/2011',
  '10/19/2007',
  '9/20/2013',
  '2/11/2016',
  '9/12/2010',
  '10/25/2008',
  '11/5/2007',
  '1/25/201

In [28]:
filled_list_wmd[0].build_df()

In [29]:
''' 
Checking data is filling in.  Not all fields will fill becuase of gaps 
in user input or differences between webMD and drugs.com fields
'''
filled_list_wmd[0].df

,ageRange,ageRange_str,comment,condition,drugName,ease_of_use,effectiveness,genRating,gender,medDuration,medDuration_str,reviewDate,role,satisfaction,site,upVotes,userName
0,"[45, 54]","[45, 54]",I began taking this for depression and pain as...,None,Cymbalta,4.0,4.0,None,None,"[24, 60]","[24, 60]",11/19/2017,Patient,4.0,WebMD,5,angie
1,"[35, 44]","[35, 44]",I was on Cymblata for roughly 2.5 years. At fi...,None,Cymbalta,3.0,2.0,None,Male,"[24, 60]","[24, 60]",8/18/2017,Patient,1.0,WebMD,20,Eldo
2,None,None,It has really helped my depression and joint a...,None,Cymbalta,4.0,4.0,None,None,"[12, 24]","[12, 24]",7/27/2017,Patient,4.0,WebMD,18,Anonymous
3,"[55, 64]","[55, 64]",Without this medicine I become suicidal.,None,Cymbalta,5.0,5.0,None,Male,"[60, 120]","[60, 120]",6/15/2017,Patient,5.0,WebMD,22,CCR
4,"[25, 34]","[25, 34]","very effective, almost saved my life. I now do...",None,Cymbalta,5.0,5.0,None,None,"[12, 24]","[12, 24]",6/12/2017,Patient,5.0,WebMD,21,caden
5,"[25, 34]","[25, 34]",Over the past several years I have tried diffe...,None,Cymbalta,5.0,5.0,None,None,"[1, 6]","[1, 6]",2/20/2017,Patient,5.0,WebMD,45,AZguy28
6,"[25, 34]","[25, 34]",I am on 90mg daily. I have Generalized Anxiety...,None,Cymbalta,3.0,3.0,None,None,"[12, 24]","[12, 24]",2/11/2017,Patient,3.0,WebMD,20,Amanda F
7,None,None,This medication put me in the hospital because...,None,Cymbalta,4.0,1.0,None,None,"[1, 6]","[1, 6]",1/19/2017,Patient,1.0,WebMD,16,Anonymous
8,"[25, 34]","[25, 34]",I am a 33 year old female recently prescribed ...,None,Cymbalta,3.0,5.0,None,Female,"[0, 1]","[0, 1]",12/30/2016,Patient,5.0,WebMD,32,In Recovery
9,None,None,I have been on this for several years and it w...,None,Cymbalta,5.0,5.0,None,None,"[24, 60]","[24, 60]",11/21/2016,Patient,5.0,WebMD,14,Anonymous


In [30]:
df_md = filled_list_wmd[0].slice_data('satisfaction', 1)

In [31]:
# testing slicing data for particular subsets
df_md

,ageRange,ageRange_str,comment,condition,drugName,ease_of_use,effectiveness,genRating,gender,medDuration,medDuration_str,reviewDate,role,satisfaction,site,upVotes,userName
1,"[35, 44]","[35, 44]",I was on Cymblata for roughly 2.5 years. At fi...,None,Cymbalta,3.0,2.0,None,Male,"[24, 60]","[24, 60]",8/18/2017,Patient,1.0,WebMD,20,Eldo
7,None,None,This medication put me in the hospital because...,None,Cymbalta,4.0,1.0,None,None,"[1, 6]","[1, 6]",1/19/2017,Patient,1.0,WebMD,16,Anonymous
11,None,None,The whole time I was on it I felt like my body...,None,Cymbalta,5.0,2.0,None,None,"[1, 6]","[1, 6]",10/21/2016,Patient,1.0,WebMD,18,Anonymous
12,"[65, 74]","[65, 74]",I weaned from Effexor XR (previously on 225 mg...,None,Cymbalta,1.0,1.0,None,None,"[0, 1]","[0, 1]",10/18/2016,Patient,1.0,WebMD,29,Loracs
22,"[25, 34]","[25, 34]",This medication caused be to become extremely ...,None,Cymbalta,5.0,1.0,None,Female,"[0, 1]","[0, 1]",1/22/2016,Patient,1.0,WebMD,26,AveMaria02
23,"[55, 64]","[55, 64]",This is awful medication. After just a short t...,None,Cymbalta,1.0,1.0,None,Female,"[1, 6]","[1, 6]",12/27/2015,Patient,1.0,WebMD,40,sunshine77
27,"[45, 54]","[45, 54]",I had short period of time when this medicatio...,None,Cymbalta,3.0,1.0,None,Female,"[12, 24]","[12, 24]",10/19/2015,Patient,1.0,WebMD,63,PoisonControl
30,"[55, 64]","[55, 64]","Maybe I haven't been on it long enough, but th...",None,Cymbalta,5.0,2.0,None,Female,"[0, 1]","[0, 1]",8/19/2015,Patient,1.0,WebMD,27,Nightowl
33,"[25, 34]","[25, 34]",Don't know why this drug didn't work for me bu...,None,Cymbalta,1.0,1.0,None,Female,"[6, 12]","[6, 12]",5/26/2015,Patient,1.0,WebMD,50,Myka34
34,"[55, 64]","[55, 64]","Right away noticed decrease in pain; however, ...",None,Cymbalta,3.0,1.0,None,Female,"[0, 1]","[0, 1]",4/12/2015,Patient,1.0,WebMD,29,mariner


In [32]:
#check 
mini_drug_list = pickle.load(open('drug_list_wmd_mini.p', 'rb'))
mini_drug_list[0].reviews[1].__dict__

{'ageRange': [35, 44],
 'ageRange_str': '[35, 44]',
 'comment': 'I was on Cymblata for roughly 2.5 years. At first it helped quite a bit helpful with my depression. It was initially not helpful for my anxiety. \n\nSlowly it became less effective. Increasing the dose made my anxiety worse. \n\nIn between doses the withdrawal would start to begin: brain shocks, swishing head, dissociation, panic, brain crunch, confusion, muscle pain, and body buzzing. If I missed a dose, I literally would think I was in some hellish nightmare. \n\nI began to feel as if Cymbalta was poisoning me.\n\nBy the lucky chance (experience of a friend), I found I could switch to Celexa. In combination with fish oil, the Cymbalta withdrawal has been minor. My mood and clarity of mind have been getting better daily. My anxiety is also going down.  ',
 'condition': None,
 'drugName': 'Cymbalta',
 'ease_of_use': 3.0,
 'effectiveness': 2.0,
 'genRating': None,
 'gender': 'Male',
 'medDuration': [24, 60],
 'medDuration_